In [1]:
import numpy as np

from sklearn.metrics import accuracy_score
from keras.datasets import reuters
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
# parameters for data load
num_words = 30000
maxlen = 50
test_split = 0.3

In [17]:
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [18]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = num_words, maxlen = maxlen, test_split = test_split)

In [19]:
# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, padding = 'post')
X_test = pad_sequences(X_test, padding = 'post')

X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

y_data = np.concatenate((y_train, y_test))
y_data = to_categorical(y_data)
y_train = y_data[:1395]
y_test = y_data[1395:]

## 1. Deep RNN
- RNNs can be made deep, with multiple layers, like CNNs or MLPs
- Beware that RNNs take long to train compared to CNNs

In [20]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

In [21]:
def deep_lstm():
    model = Sequential()
    model.add(LSTM(20, input_shape = (49,1), return_sequences = True))
    model.add(LSTM(20, return_sequences = True))
    model.add(LSTM(20, return_sequences = True))
    model.add(LSTM(20, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [22]:
model = KerasClassifier(build_fn = deep_lstm, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)

Epoch 1/200
1395/1395 [==============================] - 5s 4ms/step - loss: 3.1733 - acc: 0.6710
Epoch 2/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.8445 - acc: 0.7147
Epoch 3/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.2751 - acc: 0.7147
Epoch 4/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.2055 - acc: 0.7147
Epoch 5/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.1880 - acc: 0.7147
Epoch 6/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.1793 - acc: 0.7147
Epoch 7/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.1735 - acc: 0.7147
Epoch 8/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.1704 - acc: 0.7147
Epoch 9/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.1668 - acc: 0.7147
Epoch 10/200
1395/1395 [==============================] - 4s 3ms/step - loss: 1.1643 - acc: 0.7147
Epoch 11/200
1395/1

In [23]:
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

599/599 [==============================] - 1s 1ms/step


In [24]:
print(accuracy_score(y_pred, y_test_))

0.8330550918196995


## 2. Bidirectional RNN
- Bidirectional RNNs consider not only one-way influence of sequence, but also the other way
- Actually, they can be thought as building two separate RNNs, and merging them\
<br>
<img src="http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/09/bidirectional-rnn.png" style="width: 400px"/>
</br>

In [25]:
from keras.layers import Bidirectional

In [26]:
def bidirectional_lstm():
    model = Sequential()
    model.add(Bidirectional(LSTM(20, return_sequences = False), input_shape = (49,1)))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [27]:
model = KerasClassifier(build_fn = bidirectional_lstm, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)

Epoch 1/200
1395/1395 [==============================] - 3s 2ms/step - loss: 3.4156 - acc: 0.4416
Epoch 2/200
1395/1395 [==============================] - 2s 1ms/step - loss: 2.4331 - acc: 0.7147
Epoch 3/200
1395/1395 [==============================] - 2s 1ms/step - loss: 1.3339 - acc: 0.7147
Epoch 4/200
1395/1395 [==============================] - 1s 1ms/step - loss: 1.1161 - acc: 0.7147
Epoch 5/200
1395/1395 [==============================] - 2s 1ms/step - loss: 1.0532 - acc: 0.7154
Epoch 6/200
1395/1395 [==============================] - 2s 1ms/step - loss: 0.9806 - acc: 0.7269
Epoch 7/200
1395/1395 [==============================] - 2s 1ms/step - loss: 0.9182 - acc: 0.7806
Epoch 8/200
1395/1395 [==============================] - 1s 1ms/step - loss: 0.8846 - acc: 0.7971
Epoch 9/200
1395/1395 [==============================] - 2s 1ms/step - loss: 0.8508 - acc: 0.8079
Epoch 10/200
1395/1395 [==============================] - 2s 1ms/step - loss: 0.8241 - acc: 0.8158
Epoch 11/200
1395/1

In [28]:
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

599/599 [==============================] - 0s 641us/step


In [29]:
print(accuracy_score(y_pred, y_test_))

0.8480801335559266


## 3. Deep Bidirectional RNN
- Bidirectional RNNs can be stacked

<img src="http://www.wildml.com/wp-content/uploads/2015/09/Screen-Shot-2015-09-16-at-2.21.51-PM-272x300.png" style="width: 300px"/>

In [30]:
def deep_bidirectional_lstm():
    model = Sequential()
    model.add(Bidirectional(LSTM(10, return_sequences = True), input_shape = (49,1)))
    model.add(Bidirectional(LSTM(10, return_sequences = True)))
    model.add(Bidirectional(LSTM(10, return_sequences = True)))
    model.add(Bidirectional(LSTM(10, return_sequences = False)))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [31]:
model = KerasClassifier(build_fn = deep_bidirectional_lstm, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)

Epoch 1/200
1395/1395 [==============================] - 9s 6ms/step - loss: 3.5370 - acc: 0.6129
Epoch 2/200
1395/1395 [==============================] - 5s 4ms/step - loss: 2.4779 - acc: 0.7147
Epoch 3/200
1395/1395 [==============================] - 5s 4ms/step - loss: 1.5653 - acc: 0.7147
Epoch 4/200
1395/1395 [==============================] - 5s 4ms/step - loss: 1.3033 - acc: 0.7147
Epoch 5/200
1395/1395 [==============================] - 5s 4ms/step - loss: 1.2168 - acc: 0.7147
Epoch 6/200
1395/1395 [==============================] - 5s 4ms/step - loss: 1.1802 - acc: 0.7147
Epoch 7/200
1395/1395 [==============================] - 5s 4ms/step - loss: 1.0806 - acc: 0.7147
Epoch 8/200
1395/1395 [==============================] - 5s 4ms/step - loss: 1.0000 - acc: 0.7147
Epoch 9/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.9581 - acc: 0.7319
Epoch 10/200
1395/1395 [==============================] - 6s 4ms/step - loss: 0.8924 - acc: 0.7943
Epoch 11/200
1395/1

In [32]:
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

599/599 [==============================] - 1s 2ms/step


In [33]:
print(accuracy_score(y_pred, y_test_))

0.8280467445742905
